In [1]:
%%sh
pygmentize sklearn-boston-housing-generic.py

#!/usr/bin/env python

import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
import joblib
import os, json

if __name__ == '__main__':
    
    config_dir = '/opt/ml/input/config'
    training_dir = '/opt/ml/input/data/training'
    model_dir = '/opt/ml/model'
    
    with open(os.path.join(config_dir, 'hyperparameters.json')) as f:
        hp = json.load(f)
        print(hp)
        normalize = hp['normalize']
        test_size = float(hp['test-size'])
        random_state = int(hp['random-state'])
        
    filename = os.path.join(training_dir, 'housing.csv')
    data = pd.read_csv(filename)
    labels = data[['medv']]
    samples = data.drop(['medv'], axis=1)
    X_train, X_test, y_train, y_test = train_test_split(samples, labels, 
                                                        test_size=test_size, random_state=random_state)
    regr = LinearRegr

In [2]:
%%sh
cat Dockerfile-generic

FROM python:3.7

RUN pip3 install --no-cache scikit-learn numpy pandas joblib
RUN pip3 install --no-cache flask

COPY sklearn-boston-housing-generic.py /usr/bin/train
COPY sklearn-boston-housing-serve.py /usr/bin/serve

RUN chmod 755 /usr/bin/train /usr/bin/serve

EXPOSE 8080


In [3]:
%%sh
aws ecr create-repository --repository-name sklearn-custom


An error occurred (RepositoryAlreadyExistsException) when calling the CreateRepository operation: The repository with name 'sklearn-custom' already exists in the registry with id '613904931467'


In [ ]:
%%sh
export REGION=us-east-1
export ACCOUNT_ID=`aws sts get-caller-identity --query Account --output text`
docker build -t sklearn-custom:estimator -f Dockerfile-generic .
export IMAGE_ID=`docker images -q sklearn-custom:estimator`
docker tag $IMAGE_ID $ACCOUNT_ID.dkr.ecr.$REGION.amazonaws.com/sklearn-custom:estimator
aws ecr get-login-password | docker login --username AWS --password-stdin $ACCOUNT_ID.dkr.ecr.$REGION.amazonaws.com/sklearn-custom:estimator
docker push $ACCOUNT_ID.dkr.ecr.$REGION.amazonaws.com/sklearn-custom:estimator

In [ ]:
import sagemaker
from sagemaker.estimator import Estimator

session = sagemaker.Session()
account_id = session.boto_session.client('sts').get_caller_identity()['Account']
region = session.boto_session.region_name

training = 'file://.'
output = 'file://.'

role = sagemaker.get_execution_role()

sk = Estimator(
             image_name=account_id+'.dkr.ecr.'+region+'.amazonaws.com/sklearn-custom:estimator',
             role=role,
             train_instance_count=1, 
             train_instance_type='local',
             output_path=output,
             hyperparameters={
                  'normalize': True,
                  'test-size': 0.1,
                  'random-state': 123
             }
)

sk.fit({'training':training})

In [ ]:
%%sh
tar tvfz model.tar.gz

In [ ]:
sk_predictor = sk.deploy(instance_type='local',
                        initial_instance_count=1)

In [ ]:
from sagemaker.predictor import csv_serializer

test_samples = ['0.00632,18.00,2.310,0,0.5380,6.5750,65.20,4.0900,1,296.0,15.30,4.98',
                '0.02731,0.00,7.070,0,0.4690,6.4210,78.90,4.9671,2,242.0,17.80,9.14']

sk_predictor.content_type = 'text/csv'
sk_predictor.serializer = csv_serializer

response = sk_predictor.predict(test_samples)
print(response)

In [ ]:
sk_predictor.delete_endpoint()